In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json

In [2]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [3]:
# URL to find information on Coronavirus
url = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus/Harris-County-COVID-19-Confirmed-Cases"

url4 = "https://www.khou.com/section/news/health/coronavirus"

In [4]:
# Visiting the url 
browser.visit(url)

In [5]:
time.sleep(7)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# All the Texas counties
counties = soup.find('nav', attrs={'class':'feature-list'})

# getting all the spans with the information for each Texas county
rows = counties.find_all('span')




In [7]:
counter = 0

county = []
cases = []

for row in rows:
    try:
        print(rows[counter].text.strip().split('\n'))
        county.append(rows[counter].text.strip().split('\n')[0])
        cases.append(rows[counter].text.strip().split('\n')[1])
        counter = counter + 3
        
    except:
        print('failure')

['Dallas County', '367']
['Harris County', '229']
['Travis County', '160']
['Denton County', '137']
['Bexar County', '125']
['Collin County', '118']
['Tarrant County', '114']
['Fort Bend County', '101']
['Brazoria County', '52']
['Galveston County', '49']
['Montgomery County', '47']
['Brazos County', '36']
['McLennan County', '34']
['Williamson County', '34']
['El Paso County', '31']
['Lubbock County', '31']
['Bell County', '28']
['Smith County', '27']
['Nueces County', '20']
['Webb County', '19']
['Hidalgo County', '18']
['Wichita County', '15']
['Cameron County', '13']
['Ellis County', '13']
['Hays County', '13']
['Jefferson County', '12']
['Matagorda County', '10']
['Comal County', '9']
['Castro County', '8']
['Guadalupe County', '8']
['Johnson County', '8']
['Midland County', '8']
['Hockley County', '7']
['Hardin County', '6']
['Kendall County', '6']
['Washington County', '5']
['Bastrop County', '4']
['Gregg County', '4']
['Parker County', '4']
['Val Verde County', '4']
['Bowie Cou

In [8]:
df = pd.DataFrame(list(zip(county, cases)), columns =['Texas_County', 'COVID_19_Cases'])
df

,Texas_County,COVID_19_Cases
0,Dallas County,367
1,Harris County,229
2,Travis County,160
3,Denton County,137
4,Bexar County,125
...,...,...
106,Upshur County,1
107,Uvalde County,1
108,Van Zandt County,1
109,Willacy County,1


In [9]:
# Droping the word 'County'
df['Texas_County'] = df['Texas_County'].apply(lambda x : x.split(' County')[0])

In [10]:
# Revise? 
reported_cases = soup.find_all('text')[6].text
reported_cases

'2,052'

In [11]:
deaths = soup.find_all('text')[8].text
deaths

'27'

In [12]:
df_stats = pd.DataFrame({'Cases':[reported_cases], 'Deaths':[deaths]})
df_stats.to_csv('static/csv/stats.csv', index=False)

In [13]:
browser.visit(url3)

In [14]:
harris_table = browser.find_by_id('covidtable')

In [15]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [16]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [17]:
columns = ['Patient', 'Sex', 'Age_Range', 'Location','Exposure', 'Status']


In [18]:
harris_data = harris_data[1:]

In [19]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [20]:
# harris_df['Sex'] = harris_df['Sex'].apply(lambda x : x.split()[2])

In [21]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [22]:
harris_df.head()

,Patient,Sex,Age_Range,Location,Exposure,Status
0,1,Female,60 - 69,SW,History of Travel,Recovered
1,2,Male,60 - 69,SW,History of Travel,Recovered
2,3,Male,40 - 49,NW,Possible Exposure to Case,Active
3,4,Male,60 - 69,NW,History of Travel,Recovered
4,5,Male,40 - 49,NE,No Known Exposure,Active


In [23]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [24]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [25]:
df_counties = df['Texas_County'].tolist()

In [26]:
Counties_with_Cases = []

for i in df_counties:
    
    if i in All_Tex_Counties:
        Counties_with_Cases.append(i)
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Dallas---found
Harris---found
Travis---found
Denton---found
Bexar---found
Collin---found
Tarrant---found
Fort Bend---found
Brazoria---found
Galveston---found
Montgomery---found
Brazos---found
McLennan---found
Williamson---found
El Paso---found
Lubbock---found
Bell---found
Smith---found
Nueces---found
Webb---found
Hidalgo---found
Wichita---found
Cameron---found
Ellis---found
Hays---found
Jefferson---found
Matagorda---found
Comal---found
Castro---found
Guadalupe---found
Johnson---found
Midland---found
Hockley---found
Hardin---found
Kendall---found
Washington---found
Bastrop---found
Gregg---found
Parker---found
Val Verde---found
Bowie---found
Brown---found
Calhoun---found
Chambers---found
De Witt---found
Eastland---found
Hood---found
Lamar---found
Lavaca---found
Medina---found
Randall---found
Rockwall---found
Tom Green---found
Victoria---found
Walker---found
Wharton---found
Atascosa---found
Cass---found
Deaf Smith---found
Grayson---found
Grimes---found
Kaufman---found
Liberty---found
Llan

In [27]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [28]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [29]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "Texas_County"})
Geo_df

,Texas_County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
4,Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
...,...,...,...,...,...,...,...,...
106,Upshur,-94.941320,32.736283,230,48459,1.799823,0.147655,"(-94.94132, 32.736283)"
107,Uvalde,-99.762031,29.356698,232,48463,2.475666,0.376204,"(-99.762031, 29.356698)"
108,Van Zandt,-95.836486,32.563690,234,48467,2.402844,0.213777,"(-95.836486, 32.56369)"
109,Willacy,-97.643033,26.471329,245,48489,3.324169,0.155451,"(-97.643033, 26.471329)"


In [30]:
result = pd.merge(df, Geo_df, on='Texas_County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,Texas_County,COVID_19_Cases,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,367,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,229,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,160,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Denton,137,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
4,Bexar,125,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [31]:
result = result[['Texas_County', 'COVID_19_Cases', 'lat', 'long']]
result

,Texas_County,COVID_19_Cases,lat,long
0,Dallas,367,32.766537,-96.777819
1,Harris,229,29.859671,-95.397821
2,Travis,160,30.334233,-97.781947
3,Denton,137,33.205574,-97.116812
4,Bexar,125,29.448845,-98.519663
...,...,...,...,...
106,Upshur,1,32.736283,-94.941320
107,Uvalde,1,29.356698,-99.762031
108,Van Zandt,1,32.563690,-95.836486
109,Willacy,1,26.471329,-97.643033


In [32]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['Texas_County', 'COVID_19_Cases']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [33]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)

In [34]:
browser.visit(url4)

In [35]:
headlines_harris = browser.html
headline_soup = bs(headlines_harris, 'html.parser')

In [36]:
# lis = headline_soup.find_all('li', class_="headline-list-with-abstract__item")
links = headline_soup.find_all('div', class_="headline-list-with-abstract__headline")

titles = []
hrefs = []

In [37]:
recent = headline_soup.find_all('div', class_="story-snapshot-with-abstract__headline")[0]

In [38]:
recent

<div class="story-snapshot-with-abstract__headline">
<a class="story-snapshot-with-abstract__link" data-tracking="story-snapshot-with-abstract/headlineLinkClicked" href="/video/news/health/coronavirus/dps-troopers-at-houston-airports-enforcing-governors-travel-order/285-1a00ec88-9853-4f65-bb78-ea204b7bdf14">
<div class="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180" style="overflow-wrap: break-word;">
                        DPS troopers at Houston airports, enforcing governor's travel order
                    </div>
<div class="story-snapshot-with-abstract__mobile-title dot-ellipsis dot-height-135" style="overflow-wrap: break-word;">
                        DPS troopers at Houston airports, enforcing governor's travel order
                    </div>
</a>
</div>

In [39]:
recent_title = recent.find('div', class_="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180").text.strip()
recent_title

"DPS troopers at Houston airports, enforcing governor's travel order"

In [40]:
titles.append(recent_title)
hrefs.append('https://www.khou.com' + recent.a['href'])

In [41]:
links

[<div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/coronavirus-live-updates-saturday-march-28/285-724d0430-bb2a-471f-90b7-a29bb0f72af0">
                     Coronavirus live updates: US deaths double in 2 days, more than 2,000
                 </a>
 </div>, <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/trump-raises-idea-of-quarantines-affecting-ny-nj-and-conn/285-9b4bf50e-af6f-4593-a2a7-c3c69499c726">
                     Trump: No quarantine, but 'strong Travel Advisory' for Connecticut, New York, New Jersey
                 </a>
 </div>, <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headli

In [42]:

for article in links:
    
    try:
        
        title = article.find('a', class_="headline-list-with-abstract__link").text
        
        link = article.a['href']
        
        if (title and link):
            
            
            print('-------------')
            print(title)
            print(link)
            titles.append(title.strip())
            hrefs.append('https://www.khou.com' + link)
            
    except:
        print('no')

-------------

                    Coronavirus live updates: US deaths double in 2 days, more than 2,000
                
/article/news/health/coronavirus/coronavirus-live-updates-saturday-march-28/285-724d0430-bb2a-471f-90b7-a29bb0f72af0
-------------

                    Trump: No quarantine, but 'strong Travel Advisory' for Connecticut, New York, New Jersey
                
/article/news/health/coronavirus/trump-raises-idea-of-quarantines-affecting-ny-nj-and-conn/285-9b4bf50e-af6f-4593-a2a7-c3c69499c726
-------------

                    Here's how Houston-area residents can get tested for coronavirus
                
/article/news/health/coronavirus/houston-coronavirus-testing/285-c95ce8c4-f876-4d6c-8645-4922578617df
-------------

                    Map: Keeping track of Houston-area coronavirus cases
                
/article/news/health/coronavirus/houston-area-coronavirus-cases/285-192b8879-08b6-4ec0-8a0f-a0487ead1453
-------------

                    Coronavirus updates: CDC

In [43]:
latest_headlines_df = pd.DataFrame(list(zip(titles, hrefs)), columns =['Headline', 'link'])

In [44]:
latest_headlines_df

,Headline,link
0,"DPS troopers at Houston airports, enforcing go...",https://www.khou.com/video/news/health/coronav...
1,Coronavirus live updates: US deaths double in ...,https://www.khou.com/article/news/health/coron...
2,"Trump: No quarantine, but 'strong Travel Advis...",https://www.khou.com/article/news/health/coron...
3,Here's how Houston-area residents can get test...,https://www.khou.com/article/news/health/coron...
4,Map: Keeping track of Houston-area coronavirus...,https://www.khou.com/article/news/health/coron...
5,Coronavirus updates: CDC issues travel warning...,https://www.khou.com/article/news/health/coron...
6,'Thinking of you': Anonymous donor sends flowe...,https://www.khou.com/article/features/anonymou...
7,Houston-area organizations provide food giveaw...,https://www.khou.com/article/news/health/coron...
8,Saturday evening coronavirus headlines for Mar...,https://www.khou.com/video/news/health/coronav...
9,Local athlete on postponement of 2020 Summer O...,https://www.khou.com/video/sports/local-athlet...


In [45]:
latest_headlines_df.to_csv('static/csv/headlines.csv', index=False, header=True)

In [46]:
browser.quit()